In [1]:
import os
import cv2
import shutil
import deeplabcut
import numpy as np

2023-03-29 12:35:54.671461: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 12:35:54.777929: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-29 12:35:55.086589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yramakrishna/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2023-03-29 12:35:55.086641: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic l

Loading DLC 2.3.1...


In [2]:
# start_file  = '/home/yramakrishna/DeepLabCut/conda-environments/Ken_Test_Long.mp4'

In [4]:
# Load the pre-trained face detection model
model = cv2.dnn.readNetFromCaffe("deploy.prototxt.txt", "res10_300x300_ssd_iter_140000_fp16.caffemodel")

# Open the video file
cap = cv2.VideoCapture("Ken_Test_Long.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Set the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('CropGr.mp4', fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

check = False
gamma = 2


cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Loop through the frames in the video
while True:
    # Read the next frame
    ret, frame = cap.read()
    check = False
    
    # Check if the frame was successfully read
    if not ret:
        break
        
    # Resize the frame to a smaller size (for faster processing)
    small_frame = cv2.resize(frame, (300, 300))
    
    # Preprocess the frame for input to the neural network
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
    
    # Pass the blob through the neural network to detect faces
    model.setInput(blob)
    detections = model.forward()
    
    # Loop through the detections and draw rectangles around the monkey faces
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:  # Only show faces with high confidence+
            check = True
            # box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            # (startX, startY, endX, endY) = box.astype("int")
            # cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
    
    if check:
        # cv2.imshow("Monkey faces", frame)
        #Normalize the pixel values to the range [0, 1]
#         frame_normalized = frame.astype(np.float32) / 255.0

#         # Apply gamma correction
#         frame_corrected = np.power(frame_normalized, 1/gamma)

#         # Scale the pixel values back to the range [0, 255]
#         frame_scaled = (frame_corrected * 255.0).astype(np.uint8)

        out.write(frame)
    # Show the frame with monkey faces detected
    #cv2.imshow("Monkey faces", frame)
    
    # Wait for a key press
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video file and close the window
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# #cropping video and Gamma correction

# # Load the video file
# video = cv2.VideoCapture(dsv)

# # Get the frame rate and total number of frames
# fps = int(video.get(cv2.CAP_PROP_FPS))
# total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# #*********************************************

# # Set the start and end points to crop out
# start_sec = 10  # Crop out the first x seconds
# end_sec = 10 # Crop out the last y seconds

# #Set gamma correction value
# gamma = 2

# #**********************************************

# start_frame = int(start_sec * fps)
# end_frame = int((total_frames / fps) - end_sec) * fps

# # Set the video writer
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter('CropGr.mp4', fourcc, fps, (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# # Set the current frame number to the start frame
# video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

# # Loop through the frames and write them to the output video
# for i in range(start_frame, end_frame):
#     ret, frame = video.read()
#     if not ret:
#         break
    
#     # Normalize the pixel values to the range [0, 1]
#     frame_normalized = frame.astype(np.float32) / 255.0

#     # Apply gamma correction
#     frame_corrected = np.power(frame_normalized, 1/gamma)

#     # Scale the pixel values back to the range [0, 255]
#     frame_scaled = (frame_corrected * 255.0).astype(np.uint8)

#     out.write(frame_scaled)

# # Release the video objects
# video.release()
# out.release()
# cv2.destroyAllWindows()

In [5]:
os.mkdir("final_video")

In [6]:
shutil.move('CropGr.mp4', 'final_video/CropGr.mp4')

'final_video/CropGr.mp4'

In [7]:
os.chdir('final_video')

In [8]:
dsv = deeplabcut.DownSampleVideo('CropGr.mp4', width=320, height=240)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [9]:
try:
    os.remove('/home/yramakrishna/DeepLabCut/conda-environments/final_video/CropGr.mp4')
except OSError as e:
    # If it fails, inform the user.
    print("Error: %s - %s." % (e.filename, e.strerror))

In [10]:

file = dsv
bodyparts = ['RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'RightEye_Pupil', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'LeftEye_Pupil',
             'OutlineTop_Mid','RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']
videotype = os.path.splitext(file)[-1].lstrip('.')  # or MOV, or avi, whatever you uploaded!
video_down = file

model_options = deeplabcut.create_project.modelzoo.Modeloptions
model_selection = 'primate_face'



In [11]:
project_name = 'DLC_GazeXBI'
your_name = 'anc'

config_path, train_config_path = deeplabcut.create_pretrained_project(
    project_name,
    your_name,
    video_down,
    videotype=videotype,
    model=model_selection,
    analyzevideo=True,
    createlabeledvideo=False,
    copy_videos=True,
)



Created "/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/videos"
Created "/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/labeled-data"
Created "/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/training-datasets"
Created "/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/dlc-models"
1  videos from the directory . were added to the project.
Copying the videos
/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/videos/CropGrdownsampled.mp4
Generated "/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/config.yaml"

A new project with name DLC_GazeXBI-anc-2023-03-29 is created at /home/yramakrishna/DeepLabCut/conda-environments/final_video and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 O

/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/dlc-models/iteration-0/DLC_GazeXBIMar29-trainset95shuffle1/train/pose_cfg.yaml
Analyzing video...
Using snapshot-1030000 for model /home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/dlc-models/iteration-0/DLC_GazeXBIMar29-trainset95shuffle1


/home/yramakrishna/anaconda3/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2023-03-29 12:41:22.732242: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-29 12:41:22.745967: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-29 12:41:22.746136: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-29 12:41:22.746473: I tensorflow/core/platfo

Analyzing all the videos in the directory...
Starting to analyze %  /home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/videos/CropGrdownsampled.mp4
Loading  /home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/videos/CropGrdownsampled.mp4
Duration of video [s]:  181.14 , recorded with  29.0 fps!
Overall # of frames:  5253  found with (before cropping) frame dimensions:  320 240
Starting to extract posture


  0%|                                                  | 0/5253 [00:00<?, ?it/s]2023-03-29 12:41:24.259399: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8800
2023-03-29 12:41:24.634053: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
100%|██████████████████████████████████████| 5253/5253 [00:09<00:00, 581.70it/s]


Saving results in /home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/videos...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


In [12]:
# deeplabcut.analyze_videos(config_path, 'CropGr.mp4', save_as_csv=True, dynamic=(True,.6,30))

In [13]:
edits = {
    'dotsize': 1.5,  # size of the dots!
    'pcutoff': 0.4,  # the higher, the more conservative the plotting!
}
deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)

project_path = os.path.dirname(config_path)
full_video_path = os.path.join(
    project_path,
    'videos',
    os.path.basename(video_down),
)

# filter predictions (should already be done above ;):
deeplabcut.filterpredictions(config_path, full_video_path, videotype=videotype)

# re-create the video with your edits!
# deeplabcut.CropVideo(config_path, full_video_path, 'crop', 

deeplabcut.create_labeled_video(config_path, full_video_path, videotype=videotype, displayedbodyparts=bodyparts, draw_skeleton = True,
                                filtered=True)


Filtering with median model /home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/videos/CropGrdownsampled.mp4
Saving filtered csv poses!
Starting to process video: /home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/videos/CropGrdownsampled.mp4
Loading /home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/videos/CropGrdownsampled.mp4 and data.
Duration of video [s]: 181.14, recorded with 29.0 fps!
Overall # of frames: 5253 with cropped frame dimensions: 320 240
Generating frames and creating video.


100%|█████████████████████████████████████| 5253/5253 [00:04<00:00, 1050.80it/s]


[True]

In [14]:
shutil.move('/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-29/videos/CropGrdownsampledDLC_resnet50_DLC_GazeXBIMar29shuffle1_1030000_filtered_labeled.mp4', '/home/yramakrishna/DeepLabCut/conda-environments/CropGrdownsampledDLC_resnet50_DLC_GazeXBIMar29shuffle1_1030000_filtered_labeled.mp4')



'/home/yramakrishna/DeepLabCut/conda-environments/CropGrdownsampledDLC_resnet50_DLC_GazeXBIMar29shuffle1_1030000_filtered_labeled.mp4'

In [15]:
try:
    shutil.rmtree('/home/yramakrishna/DeepLabCut/conda-environments/final_video')
except OSError as e:
    # If it fails, inform the user.
    print("Error: %s - %s." % (e.filename, e.strerror))

In [16]:
try:
    os.remove('/home/yramakrishna/DeepLabCut/conda-environments/Ken_Test_Longdownsampled.mp4')
except OSError as e:
    # If it fails, inform the user.
    print("Error: %s - %s." % (e.filename, e.strerror))

Error: /home/yramakrishna/DeepLabCut/conda-environments/Ken_Test_Longdownsampled.mp4 - No such file or directory.


In [17]:
import tkinter as tk
from tkVideoPlayer import TkinterVideo

root = tk.Tk()

videoplayer = TkinterVideo(master=root, scaled=True)
videoplayer.load(r"/home/yramakrishna/DeepLabCut/conda-environments/CropGrdownsampledDLC_resnet50_DLC_GazeXBIMar29shuffle1_1030000_filtered_labeled.mp4")
videoplayer.pack(expand=True, fill="both")

videoplayer.play() # play the video

root.mainloop()

In [18]:
# cap = cv2.VideoCapture("/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4")
# ret, frame = cap.read()
# while(1):
#    ret, frame = cap.read()
#    cv2.imshow('frame',frame)
#    if cv2.waitKey(1) & 0xFF == ord('q') or ret==False :
#        cap.release()
#        cv2.destroyAllWindows()
#        break
#    cv2.imshow('frame',frame)

In [19]:
# from moviepy.editor import *
# import pygame
 
# clip = VideoFileClip('/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4')
# clip.preview()
 
# pygame.quit()

In [20]:
# from tkinter import *
# from tkvideo import tkvideo

# root = Tk()
# my_label = Label(root)
# my_label.pack()
# player = tkvideo("/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4", my_label, loop = 1, size = (1280,720))
# player.play()

# root.mainloop()

In [21]:
# dlc_model = config_path
# dlc_config = deeplabcut.load_config(config_file)
# dlc_model = deeplabcut.getModel(dlc_config)


# # Load the video file
# cap = cv2.VideoCapture(video_down)

# fps = int(cap.get(cv2.CAP_PROP_FPS))

# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter('CropMFace.mp4', fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# # Loop over each frame in the video
# frame_count = 0
# video.set(cv2.CAP_PROP_POS_FRAMES, 0)

# while(cap.isOpened()):
#     ret, frame = cap.read()

#     # Break the loop when the video ends
#     if not ret:
#         break

#     # Detect the monkey face using the DeepLabCut model
#     dlc_coordinates, _ = dlc_model.predict(frame, tracking=True)
#     monkey_face_coordinates = dlc_coordinates['monkey_face']

#     # Check if monkey face is detected in the frame
#     if monkey_face_coordinates.any():
#         out.write(frame_scaled)
#         # Get the bounding box coordinates of the monkey face
#         x, y, w, h = cv2.boundingRect(monkey_face_coordinates.astype(int))

#         # Crop out the monkey face and save it to a new file
#         monkey_face = frame[y:y+h, x:x+w]
#         cv2.imwrite(f'monkey_face_{frame_count}.jpg', monkey_face)
#         frame_count += 1


#     # Break the loop if 'q' is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the video capture and close all windows
# out.release()
# cap.release()
# cv2.destroyAllWindows()


In [22]:

# import cv2
# import numpy as np
# import deeplabcut

# # Load the DeepLabCut model
# config_file = 'path/to/config.yaml'
# dlc_model = deeplabcut.load_model(config_file)

# # Load the video file
# cap = cv2.VideoCapture('path/to/video')

# # Loop over each frame in the video
# frame_count = 0
# while(cap.isOpened()):
#     ret, frame = cap.read()
    
#     # Break the loop when the video ends
#     if not ret:
#         break
    
#     # Detect the monkey face using the DeepLabCut model
#     dlc_coordinates, _ = dlc_model.predict(frame, tracking=True)
#     monkey_face_coordinates = dlc_coordinates['monkey_face']
    
#     # Check if monkey face is detected in the frame
#     if monkey_face_coordinates.any():
#         # Get the bounding box coordinates of the monkey face
#         x, y, w, h = cv2.boundingRect(monkey_face_coordinates.astype(int))
        
#         # Crop out the monkey face and save it to a new file
#         monkey_face = frame[y:y+h, x:x+w]
#         cv2.imwrite(f'monkey_face_{frame_count}.jpg', monkey_face)
#         frame_count += 1
    
#     # Break the loop if 'q' is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the video capture and close all windows
# cap.release()
# cv2.destroyAllWindows()


SyntaxError: invalid non-printable character U+00A0 (3415417350.py, line 16)

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/home/yramakrishna/anaconda3/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/yramakrishna/anaconda3/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yramakrishna/anaconda3/lib/python3.9/site-packages/tkVideoPlayer/tkvideoplayer.py", line 173, in _load
    self.event_generate("<<FrameGenerated>>")
  File "/home/yramakrishna/anaconda3/lib/python3.9/tkinter/__init__.py", line 1858, in event_generate
    self.tk.call(args)
RuntimeError: main thread is not in main loop
